In [9]:
import sqlite3
import pandas as pd
import plotly.express as px
import sys
from plotly.subplots import make_subplots
import plotly.graph_objects as go

sys.path.append('../')
from functions_env import DB_PATH


conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

In [10]:
# Load individuals Main Informations
individuals = pd.read_sql_query("SELECT * FROM individuals_kept", conn)

In [11]:
# Load identifiers meta-data
df_identifiers = pd.read_sql_query("SELECT * FROM identifiers", conn)
df_identifiers['country_name'][df_identifiers['country_name'].isna()]=''
df_identifiers['identifier_name'] = df_identifiers.apply(lambda x : x['identifier_name'] + ' ('  + x['country_name'] + ')' if x['country_name']!='' else x['identifier_name'], axis=1)
df_identifiers = df_identifiers.drop(['country_wikidata_id', 'count_records', 'identifier_url'], axis=1).drop_duplicates()
df_identifiers = df_identifiers.drop(['country_name', 'identifier_name_country'], axis=1).drop_duplicates()

/var/folders/lx/9b13r6ln63jb54_3xlf9b_380000gn/T/ipykernel_67524/4189010349.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_identifiers['country_name'][df_identifiers['country_name'].isna()]=''


In [12]:
# Load infirmation about individuals and identifiers
df_ind_identifiers = pd.read_sql_query("SELECT * FROM individual_identifiers", conn)
df_ind_identifiers = df_ind_identifiers.drop('identifier_name', axis=1)
df_ind_identifiers = pd.merge(df_ind_identifiers, df_identifiers, on = 'identifiers_wikidata_id', how = 'left')

# Freebase is now mixed with Google ID
df_ind_identifiers['identifier_name'][df_ind_identifiers['identifier_name']=='Freebase ID'] = 'Google Knowledge Graph ID'
df_ind_identifiers['identifiers_wikidata_id'][df_ind_identifiers['identifiers_wikidata_id']=='P646'] = 'P2671'

# Mix the two GND
df_ind_identifiers['identifier_name'][df_ind_identifiers['identifier_name']=='Deutsche Biographie (GND) ID (Germany)'] = 'GND ID (Germany)'
df_ind_identifiers['identifiers_wikidata_id'][df_ind_identifiers['identifiers_wikidata_id']=='P7902'] = 'P227'

# Clean wrong objects
df_ind_identifiers = df_ind_identifiers[~df_ind_identifiers['individual_name'].str.contains('Painter')]

# Remove superceded
df_ind_identifiers = df_ind_identifiers[~df_ind_identifiers['identifier_name'].str.contains('superceded')]
df_ind_identifiers = df_ind_identifiers[~df_ind_identifiers['identifier_name'].str.contains('obsolete')]

# Drop duplicated after mergeing GND and Freebase
df_ind_identifiers = df_ind_identifiers.drop_duplicates()

In [13]:
df_final = pd.merge(df_ind_identifiers, individuals, on = 'individual_wikidata_id')
df_stats = df_final[['individual_wikidata_id', 'identifier_name']].drop_duplicates()

In [14]:
len(set(df_stats.individual_wikidata_id))

173017

In [15]:
# Group by 'identifier_name' and calculate the count and percentage
grouped = df_stats.groupby('identifier_name').size().reset_index(name='count')
grouped = grouped.sort_values('count', ascending=False).reset_index(drop=True)
grouped['percent'] = round(grouped['count']/len(individuals)*100, 1)
grouped.columns = ['Catalog', 'N Individuals', '%']

grouped['Catalog'] = grouped['Catalog'].str.replace('United States of America', 'US')
grouped.head(20)

,Catalog,N Individuals,%
0,VIAF ID,139198,80.5
1,Google Knowledge Graph ID,127136,73.5
2,GND ID (Germany),100841,58.3
3,ISNI,99399,57.5
4,CERL Thesaurus ID,78983,45.7
5,Library of Congress authority ID (US),76335,44.1
6,IdRef ID (France),53717,31.0
7,Nationale Thesaurus voor Auteurs ID (Netherlands),51121,29.5
8,Bibliothèque nationale de France ID (France),50378,29.1
9,FAST ID,45324,26.2
